In [2]:
pip install opencv-python numpy


In [ ]:
from PIL import Image

def is_image_long_enough(image_path, video_duration_seconds):
    try:
        # Open the image file
        with Image.open(image_path) as img:
            # Get the dimensions of the image
            width, height = img.size

            # Assuming a standard frame rate of 30 frames per second
            # Calculate the minimum height required to hold the video
            required_height = video_duration_seconds * 30

            # Check if the image is long enough
            return height >= required_height
    except Exception as e:
        print(f"Error: {e}")
        return False

# Example usage
image_path = "/content/drive/MyDrive/OIP.jpeg"
video_duration_seconds = 10  # Adjust this based on your video duration
result = is_image_long_enough(image_path, video_duration_seconds)

if result:
    print("The image is long enough to hold the video.")
else:
    print("The image is not long enough to hold the video.")

In [ ]:
import cv2
import numpy as np
from google.colab import files

def video_to_binary(video_path, resize_factor=0.5):
    video = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = video.read()
        if not ret:
            break

        # Resize the frame
        frame = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)

        # Convert each frame to binary
        frame_binary = ''.join(format(int(pixel), '08b') for pixel in frame.flatten())
        frames.append(frame_binary)

    video.release()
    return frames

def embed_video_into_image(video_frames, cover_image_path, output_path):
    cover_image = cv2.imread(cover_image_path)

    if cover_image is None:
        print(f"Error: Unable to load the cover image from {cover_image_path}")
        return

    # Embed the entire video as a binary sequence
    binary_sequence = ''.join(video_frames)
    binary_sequence = binary_sequence + '1111111111111110'  # Add a delimiter to mark the end

    index = 0
    for i in range(cover_image.shape[0]):
        for j in range(cover_image.shape[1]):
            for c in range(3):
                # Embed video frames using LSB substitution
                if index < len(binary_sequence):
                    cover_image[i, j, c] = int(format(cover_image[i, j, c], '08b')[:-1] + binary_sequence[index], 2)
                    index += 1

    # Save the modified cover image
    cv2.imwrite(output_path, cover_image)

def extract_video_from_image(manipulated_image_path, resize_factor=0.5):
    manipulated_image = cv2.imread(manipulated_image_path)

    if manipulated_image is None:
        print(f"Error: Unable to load the manipulated image from {manipulated_image_path}")
        return

    binary_sequence = ''

    for i in range(manipulated_image.shape[0]):
        for j in range(manipulated_image.shape[1]):
            for c in range(3):
                # Extract the LSBs from the manipulated image
                binary_sequence += format(manipulated_image[i, j, c], '08b')[-1]

    # Find the delimiter and extract the video frames
    delimiter_index = binary_sequence.find('1111111111111110')
    video_frames = binary_sequence[:delimiter_index]

    # Reshape the frames into BGR images
    width, height, _ = cv2.imread(manipulated_image_path).shape
    expected_size = width * height * 3

    # Ensure that the length of the frames is a multiple of the expected size
    excess = len(video_frames) % expected_size
    if excess > 0:
        video_frames = video_frames[:-excess]

    reconstructed_video = [np.array(video_frames[i:i+expected_size]).reshape((width, height, 3)) for i in range(0, len(video_frames), expected_size)]

    # Upsample the frames
    reconstructed_video = [cv2.resize(np.array(frame, dtype=np.uint8), (0, 0), fx=1/resize_factor, fy=1/resize_factor) for frame in reconstructed_video]

    return reconstructed_video

def save_reconstructed_video(video_frames, output_path='reconstructed_video.mp4'):
    # Save the reconstructed video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (video_frames[0].shape[1], video_frames[0].shape[0]))

    for frame in video_frames:
        out.write(frame)

    out.release()

# Assuming you have the paths to the video and cover image
video_path = '/content/drive/MyDrive/video (1080p).mp4'
cover_image_path = '/content/drive/MyDrive/OIP.jpeg'

# Specify the output path
output_path = '/content/drive/MyDrive/output_cover_image_with_embedded_video.jpg'

# Set resize factor
resize_factor = 0.5

# Convert video to binary frames
video_frames = video_to_binary(video_path, resize_factor)

# Embed video frames into the cover image
embed_video_into_image(video_frames, cover_image_path, output_path)

# Extract video frames from the manipulated image
reconstructed_video_frames = extract_video_from_image(output_path, resize_factor)

# Save and display the reconstructed video
save_reconstructed_video(reconstructed_video_frames, output_path='/content/drive/MyDrive/reconstructed_video.mp4')


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow  # Required for displaying images in Colab
from IPython.display import display, HTML

def display_images_and_videos(original_image_path, original_video_path, embedded_image_path, reconstructed_video_frames):
    # Display the original image
    original_image = cv2.imread(original_image_path)
    cv2_imshow(original_image)
    display(HTML("<h3>Original Image</h3>"))

    # Display the original video
    original_video = cv2.VideoCapture(original_video_path)
    frames = []
    while True:
        ret, frame = original_video.read()
        if not ret:
            break
        frames.append(frame)
    original_video.release()
    cv2_imshow(frames[0])
    display(HTML("<h3>Original Video</h3>"))

    # Display the embedded video image
    embedded_image = cv2.imread(embedded_image_path)
    cv2_imshow(embedded_image)
    display(HTML("<h3>Embedded Video Image</h3>"))

    # Display the reconstructed video frames
    for frame in reconstructed_video_frames:
        cv2_imshow(frame)
    display(HTML("<h3>Reconstructed Video</h3>"))

# Assuming you have the paths to the video, cover image, and manipulated image
original_video_path = '/content/drive/MyDrive/video (1080p).mp4'
cover_image_path = '/content/drive/MyDrive/OIP.jpeg'
embedded_image_path = '/content/drive/MyDrive/output_cover_image_with_embedded_video.jpg'

# Specify the output path
output_path = '/content/drive/MyDrive/output_cover_image_with_embedded_video.jpg'

# Convert video to binary frames
video_frames = video_to_binary(original_video_path)

# Embed video frames into the cover image
embed_video_into_image(video_frames, cover_image_path, embedded_image_path)

# Extract video frames from the manipulated image
reconstructed_video_frames = extract_video_from_image(embedded_image_path)

# Display the images and videos
display_images_and_videos(cover_image_path, original_video_path, embedded_image_path, reconstructed_video_frames)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')